# Training the LSTM emulator
The data from 01Generate_MIMO.ipynb is used to train an LSTM network. This model uses the previous 15 seconds of sensor data from the controller run and uses it to predict the heater output that the controller would use. The model is saved for use in other notebooks. Note that since the original dataset only had data points every 3 seconds, this means only 5 datapoints are used for the 15 second interval. 

Notable features:
* The model, since it is more complex, requires more inputs and generates more outputs. Feature selection with SelectKBest method (https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) indicates that the only features necessary for good model development are the setpoint and error of both temperatures. This is shown for both heater values. The actual temperature does not play as large of a roll in predicting the heater output that the controller would select. Additional features could in theory be derived and tested.
* Note that the model returns two values, one for each heater. 
* The hyperparameters used to train the model (layers, dropout, batch size, units, and window size) are not necessarily optimal. However, later notebooks indicate that the model developed emulates the behavior of the PID controller well (future work could optimize these for an even better emulation).
* Due to the stochastic nature of model training, the model can change slightly with each time it's trained. 
* To avoid overfitting, the model stops training after the validation set loss value does not improve for 15 epochs.

In [1]:
#%% Mount Google drive if you use colab for save and load LSTM models
if 'google.colab' in str(get_ipython()):
  print('Running on Colab')
  from google.colab import drive
  drive.mount('/content/drive')
    
else:
  print('Not running on Colab')

# Install(if it hasn't been) and Import GEKKO package
try:
    from gekko import GEKKO
except:
    !pip install gekko
    # restart kernel if this doesn't import
    from gekko import GEKKO

Running on Colab
Mounted at /content/drive
     |████████████████████████████████| 12.4 MB 15.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

# For scaling, feature selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression

# For LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping
from keras.models import load_model

In [4]:
# Load training data
# filename = 'MIMO_240min'
# train = pd.read_csv(filename+'_data.csv')

train = pd.read_csv('/content/drive/MyDrive/Transformer_MPC/MPC_LSTM/MIMO_240min_data.csv')
# Scale data - simplify by scaling /100, minmaxscaler results in errors when calculating error
# (T1 - Tsp)_scaled != 0 when T1 = Tsp
data = train/100.0

# Include error
data['err1'] = data['Tsp1'] - data['T1']
data['err2'] = data['Tsp2'] - data['T2']

In [6]:
# Feature selection
# Determine input and output values for model 
X = data[['T1','Tsp1','err1','T2','Tsp2','err2']]
y1 = np.ravel(data[['Q1']])
y2 = np.ravel(data[['Q2']])

# SelectKBest feature selection
bestfeatures = SelectKBest(score_func=f_regression, k='all')

# Best features for Q1
fit1 = bestfeatures.fit(X,y1)
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.bar(x=X.columns,height=fit1.scores_)
plt.title('Q1 feature selection')

# Best features for Q2
fit2 = bestfeatures.fit(X,y2)
plt.subplot(122)
plt.bar(x=X.columns,height=fit2.scores_)
plt.title('Q2 feature selection');

ValueError: ignored

In [7]:
# Hyperparameters for model
window = 20
layers = 1
batch_size = 100
drop = 0.05
units = 100

In [8]:
inp = data[['Tsp1','err1','Tsp2','err2']].values
out = data[['Q1','Q2']].values

# Load historic values of length 'window' and next real value for model
# Each time step uses last 'window' number of changes to predict the next change
X = []
Y = []
for i in range(window,len(data)):
#     X.append(inp[i-window:i,:])
#     Y.append(out[i,0])
    X.append(inp[i-window:i])
    Y.append(out[i])

# Reshape data to format accepted by LSTM
X, Y = np.array(X), np.array(Y)

In [9]:
# Keras LSTM model
model = Sequential()

if layers == 1:
    model.add(LSTM(units=units, input_shape=(X.shape[1],X.shape[2])))
    model.add(Dropout(rate=drop))
else:
    # First layer specifies input_shape and returns sequences
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X.shape[1],X.shape[2])))
    model.add(Dropout(rate=drop))
    # Middle layers return sequences
    for i in range(layers-2):
        model.add(LSTM(units=units,return_sequences=True))
        model.add(Dropout(rate=drop))
    # Last layer doesn't return anything
    model.add(LSTM(units=units))
    model.add(Dropout(rate=drop))

model.add(Dense(2))
model.compile(optimizer='adam', loss='mean_squared_error')

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

result = model.fit(X, Y, verbose=0, validation_split=0.2,
#                    callbacks = [es],
                   batch_size=batch_size,
                   epochs=500)

# Save model
model.save(filename+'_model.h5')

# Show results
epochs = es.stopped_epoch
plt.semilogy(result.history['loss'],label='loss')
plt.semilogy(result.history['val_loss'],label='val_loss')
plt.legend();

NameError: ignored